In [ ]:
import os
import json
import sys
import torch
import random
import numpy as np
random.seed(10)
from tqdm import tqdm
from utils import *
device="cuda:0"
%load_ext autoreload
%autoreload 2

In [ ]:
from utils import *

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "bigscience/bigscience/T0pp"
# model_path = "bigscience/bigscience/T0p"
# model_path = "bigscience/bigscience/T0"

tokenizer = AutoTokenizer.from_pretrained(model_path)
T0ppmodel = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)


In [ ]:
few_shot_examples = list(np.load("../data/demonstration.npy",allow_pickle=True))
sentence_data_path = ""
wordplay_data_list = ""
sentence_data_list = list(np.load(sentence_data_path,allow_pickle=True))
wordplay_data_list = list(np.load(wordplay_data_list,allow_pickle=True))

In [ ]:
test_data_list = sentence_data_list + wordplay_data_list
len(test_data_list)

In [ ]:
def get_sample_demo(sample):
    sample_demo = \
"""{}
Choose the most suitable option to answer the above question.
Options: 
A. {}
B. {}
C. {}
D. {}""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])
    
    return sample_demo

In [ ]:
for sample in tqdm(test_data_list):
    inputs = tokenizer.encode(get_sample_demo(sample), return_tensors="pt").to(device)
    outputs = T0ppmodel.generate(inputs)
    sample['response'] = tokenizer.decode(outputs[0])

In [ ]:
for index,item in enumerate(test_data_list):
    try:
        item['predict'] = item['response'].split('> ')[1].strip().split('<')[0].strip()
    except:
        print(index)

In [ ]:
word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play,sentence_play)

### Few-shot

In [ ]:
few_shot_examples[5]

In [ ]:
def get_fewshot_demo(examples,number):
    label_to_answer = ['A','B','C','D']
    few_shot_demon = examples[:number//2]+examples[4:4+number//2]
    fewshot = ''
    for sample in few_shot_demon:
        fewshot += \
"""
Question: {}
Options: 
A: {}
B: {}
C: {}
D: {}
Answer: {}
""".format(sample['Question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3],label_to_answer[int(sample['label'])])
    
    return fewshot

def get_single_demo(sample):
    sample_demo = \
"""
Question: {}
Options: 
A: {}
B: {}
C: {}
D: {}""".format(sample['question'],sample['choice_list'][0],sample['choice_list'][1],sample['choice_list'][2],sample['choice_list'][3])
    
    return sample_demo

In [ ]:
demonstration = """Read the following context and choose the best option to answer the question."""
demon_num = 8
demonstration = demonstration + get_fewshot_demo(few_shot_examples,demon_num)

In [ ]:
print(demonstration)

In [ ]:
for sample in tqdm(test_data_list):
    inputs = tokenizer.encode(demonstration+get_single_demo(sample), return_tensors="pt").to(device)
    outputs = T0ppmodel.generate(inputs)
    sample['response'] = tokenizer.decode(outputs[0])

In [ ]:
for index,item in enumerate(test_data_list):
    try:
        item['predict'] = item['response'].split('> ')[1].strip().split('<')[0].strip()
    except:
        print(index)

In [ ]:
word_play,sentence_play = getResultdata(test_data_list)
final_result = getSeperateResult(word_play,sentence_play)